### 导包

In [75]:
import utils.util as util
import json
import numpy as np
import utils.visualization as visual
from tqdm import tqdm

### 涉及阈值

In [76]:
DistThreh = 50
TimeThreh = 180

### 停留点类

In [77]:
class Stay_point:
    def __init__(self, center, point_list, arrive_time, leave_time):
        self.center = center
        self.point_list = point_list
        self.arrive_time = arrive_time
        self.leave_time = leave_time

### 支持函数

In [78]:
# 读取轨迹数据 并将轨迹坐标系转换为墨卡托坐标系
def load_trajectories(filename):
    raw_traj = json.load(open(filename))
    for k, v in raw_traj.items():
        raw_traj[k] = [v[0], util.transform_points_wgs84_to_mercator(v[1])]
    return raw_traj

### 停留点检测代码实现
#### 可选1. 基于距离/时间阈值的停留点检测算法
#### 可选2. 基于方向聚类的停留点检测算法

In [79]:
from math import sqrt


def extract_stay_points_based_on_threh(data):
    stay_points = []
    for key,value in tqdm(data.items()):
        timestamp = value[0]
        traj = value[1]
        i = 0
        length = len(traj)
        while i < length:
            j = i + 1
            token = 0
            while j < length:
                # distance = util.Distance(traj[i],traj[j])
                distance = sqrt((traj[i][0] - traj[j][0])**2 + (traj[i][1] - traj[j][1])** 2)
                if distance > DistThreh:
                    time_diff = timestamp[j] - timestamp[i]
                    if time_diff > TimeThreh:
                        center = np.array(traj[i : j + 1]).mean(0).tolist()
                        stay_point = Stay_point(center = center,
                                                point_list = traj[i:j+1],
                                                arrive_time = timestamp[i],
                                                leave_time = timestamp[j])
                        stay_points.append(stay_point)
                        i = j
                        token = 1
                    break
                j += 1
            if token != 1:
                i += 1

    return stay_points

In [80]:
def extract_stay_points_based_on_direction_clustering(data):
    
    
    
    
    return stay_points

## 读入数据，调用停留点识别算法

#### 数据输入
- traj_dict：轨迹dict
- key：轨迹id
- value：[[时间戳列表],[经纬度点列表]]

#### 调用停留点提取算法的输出结果
- stay_points：停留点list，内部每一项为停留点类，每个停留点包含center, point_list, arrive_time, leave_time属性

In [81]:
# 读取数据
data = load_trajectories('porto_traj.json')

In [82]:
print(type(data))

<class 'dict'>


In [83]:
# 数据样例
print(data['0'])

[[1372636858, 1372636859, 1372636869, 1372636870, 1372636871, 1372636872, 1372636873, 1372636877, 1372636878], [[-960588.13146654, 5034671.201921352], [-960590.1352173749, 5034675.193587552], [-960552.063951513, 5034817.564090121], [-960541.0433219211, 5034842.845052223], [-960549.0583252608, 5034864.134334601], [-960779.4896712671, 5035158.197331414], [-960779.4896712671, 5035159.5279534925], [-960779.4896712671, 5035156.866709527], [-960780.4915466846, 5035155.53608783]]]


In [84]:
# 停留点提取
stay_points = extract_stay_points_based_on_threh(data)

100%|██████████| 59378/59378 [00:01<00:00, 56215.52it/s]


In [85]:
stay_points[0]

In [86]:

print(stay_points[0].center)
print(stay_points[0].point_list)
print(stay_points[0].arrive_time)
print(stay_points[0].leave_time)

[-964859.6273087031, 5039253.3864017185]
[[-964849.1076168203, 5039241.405741261], [-964870.147000586, 5039265.367062175]]
1372654133
1372654434


In [87]:
# 可视化停留点中心位置
visual.visual_traj(util.transform_points_mercator_to_wgs84([i.center for i in stay_points]), 'stay_centers.json')